# Choosing office for the Gaming company

In [13]:
from pymongo import MongoClient
from pymongo import ASCENDING, DESCENDING
import requests
import pandas as pd
import numpy as np
import re
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL)
print(client)
db = client.get_database()
print(db)
cols = db.list_collections()
#print(cols)
for x in cols:
    print(x)

mongodb://localhost/companies
MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')
{'name': 'companies', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('66f0fe67-0302-4902-95b5-aabc2b0ca759')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'companies.companies'}}


In [14]:
n = re.compile("design",re.IGNORECASE)  

In [15]:
design = list(db.companies.find({"category_code": n}))
print(len(design))

4


In [16]:
for e in design:
    print(f'Name: {e["name"]} --> City: {e["offices"][0]["city"]} --> Address: {e["offices"][0]["address1"]} --> Money raised: {e["total_money_raised"]} --> Longitude: {e["offices"][0]["longitude"]} Latitude: {e["offices"][0]["latitude"]}\n')

Name: 99designs --> City: San Francisco --> Address: 447 Battery St. --> Money raised: $35M --> Longitude: -122.4005983 Latitude: 37.7955307

Name: Graticle --> City: Ellensburg --> Address: 100 E. Third Ave. #373 --> Money raised: $0 --> Longitude: -122.6931439 Latitude: 45.7968414

Name: Kickstarter --> City: Brooklyn --> Address: 58 Kent St --> Money raised: $10M --> Longitude: None Latitude: None

Name: Moonfruit --> City: London --> Address:  --> Money raised: $2.25M --> Longitude: None Latitude: None



In [17]:
df_design = pd.DataFrame(list(design))
offices = df_design.explode("offices").apply(lambda e: e.offices,axis=1,result_type="expand")
offices

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598
0,Australia,204 Wellington St.,,3065,Collingwood,None,AUS,-37.802659,144.986855
0,Europe,Schlesische Str. 29-30,,,Berlin,None,DEU,52.498620,13.446903
1,Office,100 E. Third Ave. #373,,98926,Ellensburg,WA,USA,45.796841,-122.693144
2,,58 Kent St,,11222,Brooklyn,NY,USA,NaN,NaN
3,UK Office,,,,London,None,GBR,NaN,NaN


In [18]:
clean_offices = pd.concat([df_design[["name","_id","category_code","total_money_raised"]], offices], axis=1)
clean_offices

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,99designs,52cdef7c4bab8bd675298447,design,$35M,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598
0,99designs,52cdef7c4bab8bd675298447,design,$35M,Australia,204 Wellington St.,,3065,Collingwood,None,AUS,-37.802659,144.986855
0,99designs,52cdef7c4bab8bd675298447,design,$35M,Europe,Schlesische Str. 29-30,,,Berlin,None,DEU,52.498620,13.446903
1,Graticle,52cdef7e4bab8bd67529ba4e,design,$0,Office,100 E. Third Ave. #373,,98926,Ellensburg,WA,USA,45.796841,-122.693144
2,Kickstarter,52cdef7e4bab8bd67529bd1a,design,$10M,,58 Kent St,,11222,Brooklyn,NY,USA,NaN,NaN
3,Moonfruit,52cdef7f4bab8bd67529c47a,design,$2.25M,UK Office,,,,London,None,GBR,NaN,NaN


In [19]:
def getFullAddress(df):
    full_address = df["address1"]+" "+df["city"]
    return full_address

clean_offices["full_address"] = clean_offices.apply(getFullAddress, axis=1)

In [20]:
clean_offices

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,full_address
0,99designs,52cdef7c4bab8bd675298447,design,$35M,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598,447 Battery St. San Francisco
0,99designs,52cdef7c4bab8bd675298447,design,$35M,Australia,204 Wellington St.,,3065,Collingwood,None,AUS,-37.802659,144.986855,204 Wellington St. Collingwood
0,99designs,52cdef7c4bab8bd675298447,design,$35M,Europe,Schlesische Str. 29-30,,,Berlin,None,DEU,52.498620,13.446903,Schlesische Str. 29-30 Berlin
1,Graticle,52cdef7e4bab8bd67529ba4e,design,$0,Office,100 E. Third Ave. #373,,98926,Ellensburg,WA,USA,45.796841,-122.693144,100 E. Third Ave. #373 Ellensburg
2,Kickstarter,52cdef7e4bab8bd67529bd1a,design,$10M,,58 Kent St,,11222,Brooklyn,NY,USA,NaN,NaN,58 Kent St Brooklyn
3,Moonfruit,52cdef7f4bab8bd67529c47a,design,$2.25M,UK Office,,,,London,None,GBR,NaN,NaN,London


In [21]:
def geocode(address):
    res = requests.get(f"https://geocode.xyz/{address}", params={"json":1})
    data = res.json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

In [23]:
#address = "58 Kent St Brooklyn"
#geocode(address)
clean_offices["geopoint"] = clean_offices["full_address"].apply(geocode)

In [24]:
clean_offices

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,full_address,geopoint
0,99designs,52cdef7c4bab8bd675298447,design,$35M,United States (HQ),447 Battery St.,3rd Floor,94111,San Francisco,CA,USA,37.795531,-122.400598,447 Battery St. San Francisco,"{'type': 'Point', 'coordinates': [-122.40043, ..."
0,99designs,52cdef7c4bab8bd675298447,design,$35M,Australia,204 Wellington St.,,3065,Collingwood,None,AUS,-37.802659,144.986855,204 Wellington St. Collingwood,"{'type': 'Point', 'coordinates': [-80.16536, 4..."
0,99designs,52cdef7c4bab8bd675298447,design,$35M,Europe,Schlesische Str. 29-30,,,Berlin,None,DEU,52.498620,13.446903,Schlesische Str. 29-30 Berlin,"{'type': 'Point', 'coordinates': [13.44778, 52..."
1,Graticle,52cdef7e4bab8bd67529ba4e,design,$0,Office,100 E. Third Ave. #373,,98926,Ellensburg,WA,USA,45.796841,-122.693144,100 E. Third Ave. #373 Ellensburg,"{'type': 'Point', 'coordinates': [0.0, 0.0]}"
2,Kickstarter,52cdef7e4bab8bd67529bd1a,design,$10M,,58 Kent St,,11222,Brooklyn,NY,USA,NaN,NaN,58 Kent St Brooklyn,"{'type': 'Point', 'coordinates': [-73.95878, 4..."
3,Moonfruit,52cdef7f4bab8bd67529c47a,design,$2.25M,UK Office,,,,London,None,GBR,NaN,NaN,London,"{'type': 'Point', 'coordinates': [-0.11348, 51..."


In [25]:
def getDataFrameofCity(df,ciudad):
    df_out = df[df["city"]==ciudad]
    return df_out

In [26]:
brooklyn_design = getDataFrameofCity(clean_offices,"Brooklyn")
brooklyn_design

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,full_address,geopoint
2,Kickstarter,52cdef7e4bab8bd67529bd1a,design,$10M,,58 Kent St,,11222,Brooklyn,NY,USA,NaN,NaN,58 Kent St Brooklyn,"{'type': 'Point', 'coordinates': [-73.95878, 4..."


In [27]:
tech = list(db.companies.find({"category_code": "tech"}))
print(len(tech))

0


In [28]:
clean_offices["city"]

0    San Francisco
0      Collingwood
0           Berlin
1       Ellensburg
2         Brooklyn
3           London
Name: city, dtype: object

In [29]:
tech_companies = pd.DataFrame()

In [30]:
def millionDollarTech(ciudad):
    n = re.compile("tech",re.IGNORECASE)
    match = re.compile(r'[M]')
    # Create the query
    query = {"$and":[{"offices.city": ciudad}, {"total_money_raised": match}, {"category_code": n}]}
    # Do the query. Extract name company and where is located.
    companies = (list(db.companies.find(query, {"name":1,"category_code":1,"total_money_raised":1,"offices":1})))
    return (companies)

In [31]:
for ciudad in ['San Francisco','Collingwood','Berlin','Ellensburg', 'Brooklyn','London']:
    print(ciudad,"-->", len(millionDollarTech(ciudad)))

San Francisco --> 11
Collingwood --> 0
Berlin --> 1
Ellensburg --> 0
Brooklyn --> 2
London --> 6


In [32]:
brooklyn_mill = pd.DataFrame(millionDollarTech("Brooklyn"))
brooklyn_mill

,_id,name,category_code,total_money_raised,offices
0,52cdef7e4bab8bd67529b6a8,EnergyHub,cleantech,$33.1M,"[{'description': '', 'address1': '232 3rd St.'..."
1,52cdef7f4bab8bd67529c08f,OwnEnergy,cleantech,$28.8M,"[{'description': 'HQ', 'address1': '45 Main St..."


In [33]:
brooklyn_offices = brooklyn_mill.explode("offices").apply(lambda e: e.offices,axis=1,result_type="expand")
brooklyn_offices

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,,232 3rd St.,Suite C201,11215,Brooklyn,NY,USA,NaN,NaN
1,HQ,"45 Main Street, Suite 538",,11201,Brooklyn,NY,USA,40.702847,-73.990681


In [34]:
brooklyn_tech = pd.concat([brooklyn_mill[["name","_id","category_code","total_money_raised"]], brooklyn_offices], axis=1)
brooklyn_tech

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,EnergyHub,52cdef7e4bab8bd67529b6a8,cleantech,$33.1M,,232 3rd St.,Suite C201,11215,Brooklyn,NY,USA,NaN,NaN
1,OwnEnergy,52cdef7f4bab8bd67529c08f,cleantech,$28.8M,HQ,"45 Main Street, Suite 538",,11201,Brooklyn,NY,USA,40.702847,-73.990681


In [35]:
brooklyn_tech["full_address"] = brooklyn_tech.apply(getFullAddress, axis=1)
brooklyn_tech

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,full_address
0,EnergyHub,52cdef7e4bab8bd67529b6a8,cleantech,$33.1M,,232 3rd St.,Suite C201,11215,Brooklyn,NY,USA,NaN,NaN,232 3rd St. Brooklyn
1,OwnEnergy,52cdef7f4bab8bd67529c08f,cleantech,$28.8M,HQ,"45 Main Street, Suite 538",,11201,Brooklyn,NY,USA,40.702847,-73.990681,"45 Main Street, Suite 538 Brooklyn"


In [36]:
def transformToGeoPoint(s):
    if np.isnan(s.latitude) or np.isnan(s.longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.longitude, s.latitude]
    }

In [37]:
brooklyn_tech["geopoint"] = brooklyn_tech.apply(transformToGeoPoint, axis=1)
brooklyn_tech

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,full_address,geopoint
0,EnergyHub,52cdef7e4bab8bd67529b6a8,cleantech,$33.1M,,232 3rd St.,Suite C201,11215,Brooklyn,NY,USA,NaN,NaN,232 3rd St. Brooklyn,None
1,OwnEnergy,52cdef7f4bab8bd67529c08f,cleantech,$28.8M,HQ,"45 Main Street, Suite 538",,11201,Brooklyn,NY,USA,40.702847,-73.990681,"45 Main Street, Suite 538 Brooklyn","{'type': 'Point', 'coordinates': [-73.9906813,..."


In [38]:
brooklyn_design

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,full_address,geopoint
2,Kickstarter,52cdef7e4bab8bd67529bd1a,design,$10M,,58 Kent St,,11222,Brooklyn,NY,USA,NaN,NaN,58 Kent St Brooklyn,"{'type': 'Point', 'coordinates': [-73.95878, 4..."


In [135]:
brooklyn = brooklyn_tech.append(brooklyn_design)
brooklyn

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,full_address,geopoint
0,EnergyHub,52cdef7e4bab8bd67529b6a8,cleantech,$33.1M,,232 3rd St.,Suite C201,11215,Brooklyn,NY,USA,NaN,NaN,232 3rd St. Brooklyn,None
1,OwnEnergy,52cdef7f4bab8bd67529c08f,cleantech,$28.8M,HQ,"45 Main Street, Suite 538",,11201,Brooklyn,NY,USA,40.702847,-73.990681,"45 Main Street, Suite 538 Brooklyn","{'type': 'Point', 'coordinates': [-73.9906813,..."
2,Kickstarter,52cdef7e4bab8bd67529bd1a,design,$10M,,58 Kent St,,11222,Brooklyn,NY,USA,NaN,NaN,58 Kent St Brooklyn,"{'type': 'Point', 'coordinates': [-73.95878, 4..."


In [136]:
geocode("232 3rd St. Brooklyn")

{'type': 'Point', 'coordinates': [-89.30641, 42.78376]}

In [137]:
brooklyn_close = brooklyn.dropna(subset=['geopoint'])
brooklyn_close

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,full_address,geopoint
1,OwnEnergy,52cdef7f4bab8bd67529c08f,cleantech,$28.8M,HQ,"45 Main Street, Suite 538",,11201,Brooklyn,NY,USA,40.702847,-73.990681,"45 Main Street, Suite 538 Brooklyn","{'type': 'Point', 'coordinates': [-73.9906813,..."
2,Kickstarter,52cdef7e4bab8bd67529bd1a,design,$10M,,58 Kent St,,11222,Brooklyn,NY,USA,NaN,NaN,58 Kent St Brooklyn,"{'type': 'Point', 'coordinates': [-73.95878, 4..."


In [138]:
brooklyn_close = brooklyn_close.drop(columns=['longitude','latitude'])
brooklyn_close

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,full_address,geopoint
1,OwnEnergy,52cdef7f4bab8bd67529c08f,cleantech,$28.8M,HQ,"45 Main Street, Suite 538",,11201,Brooklyn,NY,USA,"45 Main Street, Suite 538 Brooklyn","{'type': 'Point', 'coordinates': [-73.9906813,..."
2,Kickstarter,52cdef7e4bab8bd67529bd1a,design,$10M,,58 Kent St,,11222,Brooklyn,NY,USA,58 Kent St Brooklyn,"{'type': 'Point', 'coordinates': [-73.95878, 4..."


In [156]:
def getLatitudeFromGeoPoint(df):
    for e in df['geopoint']:
        if e == 'coordinates':
            latitude = df['geopoint'][e][1]
    return latitude

In [157]:
def getLongitudeFromGeoPoint(df):
    for e in df['geopoint']:
        if e == 'coordinates':
            longitude = df['geopoint'][e][0]
    return longitude

In [160]:
brooklyn_close["longitude"] = brooklyn_close.apply(getLongitudeFromGeoPoint, axis=1)
brooklyn_close

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,full_address,geopoint,latitude,longitude
1,OwnEnergy,52cdef7f4bab8bd67529c08f,cleantech,$28.8M,HQ,"45 Main Street, Suite 538",,11201,Brooklyn,NY,USA,"45 Main Street, Suite 538 Brooklyn","{'type': 'Point', 'coordinates': [-73.9906813,...",40.702847,-73.990681
2,Kickstarter,52cdef7e4bab8bd67529bd1a,design,$10M,,58 Kent St,,11222,Brooklyn,NY,USA,58 Kent St Brooklyn,"{'type': 'Point', 'coordinates': [-73.95878, 4...",40.730590,-73.958780


In [159]:
brooklyn_close["latitude"] = brooklyn_close.apply(getLatitudeFromGeoPoint, axis=1)
brooklyn_close

,name,_id,category_code,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,full_address,geopoint,latitude,longitude
1,OwnEnergy,52cdef7f4bab8bd67529c08f,cleantech,$28.8M,HQ,"45 Main Street, Suite 538",,11201,Brooklyn,NY,USA,"45 Main Street, Suite 538 Brooklyn","{'type': 'Point', 'coordinates': [-73.9906813,...",40.702847,-73.990681
2,Kickstarter,52cdef7e4bab8bd67529bd1a,design,$10M,,58 Kent St,,11222,Brooklyn,NY,USA,58 Kent St Brooklyn,"{'type': 'Point', 'coordinates': [-73.95878, 4...",40.730590,-73.958780


In [165]:
lat = []
long = []
for e in brooklyn_close['latitude']:
    lat.append(e)

for e in brooklyn_close['longitude']:
    long.append(e)

In [166]:
latitude_avg = sum(lat)/len(lat)
longitude_avg = sum(long)/len(long)
print(longitude_avg,latitude_avg)

-73.97473065 40.7167187


In [173]:
df = brooklyn_close[['name','category_code','total_money_raised','full_address','geopoint','longitude','latitude']]
df

,name,category_code,total_money_raised,full_address,geopoint,longitude,latitude
1,OwnEnergy,cleantech,$28.8M,"45 Main Street, Suite 538 Brooklyn","{'type': 'Point', 'coordinates': [-73.9906813,...",-73.990681,40.702847
2,Kickstarter,design,$10M,58 Kent St Brooklyn,"{'type': 'Point', 'coordinates': [-73.95878, 4...",-73.958780,40.730590


In [172]:
df.to_json("output/brooklyn.json",orient="records")